In [9]:
%load_ext autoreload
%load_ext tensorboard
%autoreload 2
import os
import sys
sys.path.insert(0, os.path.abspath('..'))
sys.path.insert(0, os.path.abspath('../..'))

import importlib

import contextlib

import json
import os
import random

import numpy as np
import torch
from collections import defaultdict
from utils import get_default_path
import time
from utils import Stopwatch
import datetime

import ai.stabledisco as sd
import ai.torchmodules as torchmodules
import ai.torchmodules.data as torchdata
import ai.torchmodules.utils as torchutils
import ai.stabledisco.utils as sdutils
import clip
import torch
import torch.nn as nn
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [2]:
df_path = get_default_path("large_datasets", "aug_prompts.feather")
prompt_dataframe = pd.read_feather(df_path)

In [4]:
df_path = get_default_path("sd_discord", "sd_wave2_dataframe.feather")
discord_prompt_dataframe = pd.read_feather(df_path)

In [36]:
print(len(prompt_dataframe))
print(len(rows))

3221483
1285619


In [7]:
rows = []
for row in discord_prompt_dataframe.itertuples():
    rows.append({"prompt": row.prompt,
           "text_tokens": row.text_tokens,
           "aesthetic": None})

In [32]:
prompt_dataframe = pd.concat([prompt_dataframe, pd.DataFrame.from_records(rows)], ignore_index=True)


In [12]:
print(len(rows))

1285619


In [35]:
prompt_dataframe = prompt_dataframe.reset_index(drop=True)
prompt_dataframe.drop_duplicates(subset="prompt", ignore_index=True, inplace=True)
prompt_dataframe = prompt_dataframe.sample(frac=1).reset_index(drop=True)

In [15]:
df_path = get_default_path("large_datasets", "aug_prompts.feather")
prompt_dataframe.to_feather(df_path)

In [30]:
prompt_dataframe = pd.DataFrame({
    "prompt": [],
    "cfg": [],
    "width": [],
    "height": [],
    "num_imgs": [],
    "sampler": [],
    "is_grid": [],
    "steps": [],
    "seed": [],
    "urls": [],
    "date": [],
    "user": [],
}) 

In [9]:
sd_discord = get_default_path("sd_discord2")
json_paths = [path for path in os.listdir(sd_discord) if path.endswith(".json")]
json_paths = ["DreamStudio - Art bots - art-gen [970175047891308544].json"]
messages = []

def is_prompt_message(msg):
    return msg["author"]["isBot"] and "content" in msg and msg["content"].startswith("Dreamt")

for file_name in json_paths:
    file_path = os.path.join(sd_discord, file_name)
    with open(file_path, 'r') as discord_file:
        messages += [msg for msg in json.load(discord_file)['messages'] if is_prompt_message(msg)]

In [ ]:
import re
from storage.data.command.stablediscoprompt import StableDiscoPrompt
print(len(messages))
seen = set()

def add_to_rows(rows, discord_json, dry_run=False):
    content = discord_json["content"].lower()
    multi_command_idx = content.find("The commands")
    if multi_command_idx != -1:
        content = content[:multi_command_idx]
        
    
    if not discord_json["attachments"]:
        print(f"No attachment for {content}")
        return
    
        
    content = re.sub(r"\s", " ", content)
    prompt_str = re.findall(r'(?<=!dream )".*".*-.*(?=`)', content)
    if not prompt_str:
        prompt_str = re.findall(r'(?<=!dream ").*(?=" -)',content )
    if not prompt_str:
        prompt_str = re.findall(r'(?<=dreamt of ").*(?=")',content )
    if not prompt_str:
        print(f"Failed to extract {content}")
        print()
        return
    prompt_str = prompt_str[0]
    cut_idx = re.search(r"[` ]*(\*\*|\*\*WARNING\*\*|The seeds for each individual image are)", prompt_str) 

    if cut_idx:
        cut_idx = cut_idx.span()[0]
        prompt_str = prompt_str[:cut_idx]
    
    try:
        command = StableDiscoPrompt.from_str(prompt_str) 
    except Exception as ex:
        print(f"Can't extract prompt from {prompt_str}\n full str {content}")
        print(ex)
        print()
        return
    
    prompt = command.prompt
    first_quote = prompt.find("\"")
    prompt = prompt[:first_quote] + prompt[first_quote+1:]
    
    last_quote = prompt.rfind("\"")
    prompt = prompt[:last_quote] + prompt[last_quote+1:]
    prompt = prompt.strip()
    if prompt in seen:
        return
    seen.add(prompt)
        
    
        
    row = {"prompt": prompt,
           "text_tokens": None,
           "aesthetic": None}

    rows.append(row)
    
rows = []
for idx, msg in enumerate(messages):
    if idx != 0 and idx % 100000 == 0:
        print(f"Finished {idx} of {len(messages)}")
    try:
        add_to_rows(rows, msg)
    except Exception as ex:
        print(ex)
        raise(ex)

In [ ]:
print(len())

In [ ]:
for row in discord_prompt_dataframe.iterrows():
    

In [62]:
prompt_dataframe = pd.DataFrame.from_records(rows)

In [31]:
len(prompt_dataframe.loc[prompt_dataframe['cfg'] != 7.0])

3

In [30]:
#file_path = os.path.join(sd_discord, file_name)
prompt_dataframe.to_csv(file_path)

In [ ]:
prompt_dataframe.describe()

In [39]:
prompt_dataframe['img_path'] = ""

In [40]:
print(prompt_dataframe.loc[0])

prompt                                                   moai
cfg                                                       7.0
width                                                     512
height                                                    512
num_imgs                                                    1
sampler                                                 k_lms
is_grid                                                 False
steps                                                      50
seed                                               2878504252
urls        ['https://cdn.discordapp.com/attachments/10056...
date                            2022-08-07T00:14:18.668+00:00
user                                             oliveoil2222
img_path                                                     
Name: 0, dtype: object


In [ ]:
import shutil
import requests

def save_img(url, path):
    prompt_dataframe.loc[idx,"urls"]
    start = url.find("['") + 2
    end = url.find("'",start+1)
    url = url[start:end]
    response = requests.get(url, stream=True)
    with open(path, 'wb') as out_file:
        shutil.copyfileobj(response.raw, out_file)
    del response

import requests
img_dir = "sd_discord/imgs"

for idx in prompt_dataframe.index:
    if idx % 10000 == 0:
        print(f"{idx} of {len(prompt_dataframe)} finished")
    img_path = get_default_path(img_dir, f"{idx}.png")
    save_img(prompt_dataframe.loc[idx,"urls"], img_path)
    prompt_dataframe.loc[idx, 'img_path'] = img_path

In [60]:
prompt_dataframe.to_csv(file_path)

In [59]:
print(len(prompt_dataframe[prompt_dataframe['img_path'] == ""]))

1564635
